# 從語言模型到AI Agent
# Build a Basic RAG System

研究重点为基于《红楼梦》数据集的检索增强生成（RAG）技术实现。研究目标是通过采用不同的检索模型、优化数据集预处理流程以及改进提示词格式，提升检索增强生成系统的性能表现。

# Pre: 前期准备

In [ ]:
# Download Dataset

!curl -L https://www.csie.ntu.edu.tw/~b10902031/test_public.jsonl -o res/hw2/test_public.jsonl
!curl -L https://www.csie.ntu.edu.tw/~b10902031/test_private.jsonl -o res/hw2/test_private.jsonl
!curl -L https://www.csie.ntu.edu.tw/~b10902031/hongloumeng_fulltext.txt -o res/hw2/hongloumeng_fulltext.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0   0     0   0     0     0     0  --:--:-- --:--:-- --:--:--     0
  0     0   0     0   0     0     0     0  --:--:-- --:--:-- --:--:--     0
100 12795 100 12795   0     0 10033     0   0:00:01  0:00:01 --:--:-- 10043
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0   0     0   0     0     0     0  --:--:-- --:--:-- --:--:--     0
  0     0   0     0   0     0     0     0  --:--:-- --:--:-- --:--:--     0
100  6513 100  6513   0     0  6647     0  --:--:-- --:--:-- --:--:--  6652
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0   0     0   0     0     0     0  --:--:-- --:--:-- --:--:

In [1]:
# Import necessary libraries
import os                           # 作業系統相關功能 / Operating system related functions
import json                         # 處理 JSON 格式資料的標準套件 / Standard JSON library
import torch                        # PyTorch 是一個深度學習框架 / PyTorch deep learning framework
from typing import List             # 用來註解 List 型別 / Type annotation for list
from transformers import (          # Transformers 模型相關函式庫 / HuggingFace Transformers library
    AutoModelForCausalLM,           # 自動載入因果語言模型 / Auto loader for causal language models
    AutoTokenizer,                  # 自動載入對應的 tokenizer / Auto loader for tokenizer
    BitsAndBytesConfig,             # 用來設定量化模型的參數 / Configuration for model quantization
    pipeline                        # 提供簡單的模型推論介面 / Easy interface for model inference
)
from tqdm.autonotebook import tqdm  # 顯示進度條（適用於 Jupyter Notebook） / Progress bar in Jupyter
import jieba                        # 中文斷詞工具 / Chinese word segmentation
from rank_bm25 import BM25Okapi     # BM25 搜尋模型 / BM25 retrieval model
from sentence_transformers import SentenceTransformer  # 向量模型 / Sentence embedding model
from huggingface_hub import login  # 用於 HuggingFace Hub 認證 / HuggingFace Hub authentication
import random                      # 用於隨機抽樣 / For random sampling
# 资源目录
RES_DIR = "res/hw2/"
# 输出目录
OUTPUT_DIR = "output/"

C:\Users\ROG\AppData\Roaming\Python\Python313\site-packages\jieba\_compat.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
# Device checking
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(f"PyTorch version: {torch.__version__}")

Using device: cuda
PyTorch version: 2.9.1+cu130


In [3]:
# Load Queries
queries = []
with open(RES_DIR + "test_public.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        queries.append(data["query"])
with open(RES_DIR + "test_private.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        queries.append(data["query"])
print(f"Loaded {len(queries)} queries.")

Loaded 100 queries.


In [4]:
# Load Knowledge Base and Process into Chunks 載入紅樓夢全文，並分段處理（每段長度為 chunk_size）
chunks = []                   # 存放每段文字的清單 / List to store text chunks
passage = ""                     # 存放全文的變數 / Full text content
chunk_size = 512                # 每段文字的長度上限 / Size of each chunk
stride = 512                   # 每段間的步長 / Stride between chunks
with open(RES_DIR + "hongloumeng_fulltext.txt", "r", encoding="utf-8") as f:
    passage = f.read()

length = len(passage)
for i in range(0, length, stride):
    chunk = passage[i:i + chunk_size]
    chunks.append(chunk)

print(f"Total chunks created: {len(chunks)}")

Total chunks created: 1710


In [5]:
# Load Retrieval Models 初始化兩種檢索模型：BM25 和向量模型

# 建立 BM25 模型所需的斷詞資料 / Tokenize each chunk using jieba for BM25
tokenized_chunks = [list(jieba.cut(chunk)) for chunk in chunks]
bm25 = BM25Okapi(tokenized_chunks)  # 建立 BM25 檢索模型 / Build BM25 retrieval object

# 載入語句向量模型（Embedding Model）/ Load the sentence transformer model
embedding_model = SentenceTransformer("intfloat/multilingual-e5-large")

# 對所有 chunks 計算向量 / Encode chunks into embeddings
chunks_embeddings = embedding_model.encode(chunks, show_progress_bar=True)

print("Retrieval models are ready.")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ROG\AppData\Local\Temp\jieba.cache
Loading model cost 0.298 seconds.
Prefix dict has been built successfully.


Batches:   0%|          | 0/54 [00:00<?, ?it/s]

Retrieval models are ready.


In [6]:
# Retrieval Functions

# 定義 BM25 檢索函式 / Define retrieval function using BM25
def bm25_retrieve(query: str, chunks: List[str], bm25: BM25Okapi) -> List[str]:
    tokenized_query = list(jieba.cut(query))  # 對查詢進行斷詞 / Tokenize the query
    scores = bm25.get_scores(tokenized_query) # 計算每段文字的分數 / Get relevance scores
    rank = sorted(zip(chunks, scores), key=lambda x: x[1], reverse=True)  # 依分數排序 / Rank by score
    return [chunk for chunk, _ in rank]   # 回傳排序後的段落 / Return ranked chunks

# 定義向量檢索函式 / Define retrieval function using embedding similarity
def embedding_retrieve(query: str, chunks: List[str], chunks_embeddings: List[str], embedding_model: SentenceTransformer) -> List[str]:
    query_embeddings = embedding_model.encode(query, prompt_name="query")  # 取得查詢的向量表示 / Encode query
    scores = list(embedding_model.similarity(query_embeddings, chunks_embeddings)[0])  # 計算相似度 / Compute similarities
    rank = sorted(zip(chunks, scores), key=lambda x: x[1], reverse=True)  # 排序 / Rank by score
    return [chunk for chunk, _ in rank]

# 定義 Reciprocal Rank Fusion 函式，融合多種排序 / Combine multiple rankings using RRF
def reciprocal_rank_fusion(*ranked_lists, k=60) -> List[str]:
    scores = {}
    for rl in ranked_lists:
        for rank, doc_id in enumerate(rl, start=1):  # rank 從 1 開始 / Rank is 1-based
            scores[doc_id] = scores.get(doc_id, 0.0) + 1.0 / (k + rank)

    fused = sorted(scores.items(), key=lambda x: (-x[1], x[0]))  # 排序並融合 / Sort fused scores
    return [d for d, _ in fused]

In [7]:
# Load LLM

# 設定量化參數，減少記憶體使用 / Set quantization settings for smaller memory usage
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                    # 使用 4-bit 權重 / Use 4-bit weights
    bnb_4bit_use_double_quant=True,       # 啟用雙重量化 / Enable double quantization
    bnb_4bit_quant_type="nf4",            # 使用 nf4 量化類型 / Quantization type
    bnb_4bit_compute_dtype=torch.bfloat16 # 使用 bfloat16 進行計算 / Use bfloat16 for compute
)
# 載入微調好的語言模型（Gemma）/ Load pretrained LLM
llm = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path="google/gemma-3-4b-it",
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
)
# 載入對應的 tokenizer / Load tokenizer for the model
tokenizer = AutoTokenizer.from_pretrained(
    "google/gemma-3-4b-it",
    device_map="auto",
)
# 使用 pipeline 包裝模型推論介面 / Create a pipeline for generation
llm_pipe = pipeline(
    "text-generation",           # 任務為文本生成 / Task type
    model=llm,                   # 使用的模型 / LLM
    tokenizer=tokenizer,        # tokenizer
    max_new_tokens=256,         # 回應最大長度 / Maximum new tokens
    do_sample=False,            # 不使用隨機 sampling（使用貪婪解碼）/ Greedy decoding
    temperature=None,
    top_p=None,
    top_k=None,
)

`torch_dtype` is deprecated! Use `dtype` instead!
Exception in thread Thread-5 (_readerthread):
Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\envs\pytorch\Lib\threading.py", line 1043, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "c:\ProgramData\anaconda3\envs\pytorch\Lib\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "c:\ProgramData\anaconda3\envs\pytorch\Lib\threading.py", line 994, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\envs\pytorch\Lib\subprocess.py", line 1615, in _readerthread
    buffer.append(fh.read())
                  ~~~~~~~^^
  File "<frozen codecs>", line 325, in decode
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb2 in position 7: invalid start byte


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [8]:
# Define Prompts 定義提示語（prompt），用來告訴模型角色與任務
# Define system/user prompts to guide the model's behavior

SIMPLE_SYSTEM_PROMPT = """
你是一位精通中國古典文學《紅樓夢》的專家。
請根據提問，直接提供精簡並準確的解答，避免與問題無關的內容。
""".strip()

SYSTEM_PROMPT = """
你是一位精通中國古典文學《紅樓夢》的專家。
請根據**問題**，從使用者提供的**《紅樓夢》文章片段**找出與問題相關的內容，直接提供精簡並準確的解答，避免與問題無關的內容。
""".strip()

USER_PROMPT = """
### 問題
{query}
### 《紅樓夢》文章片段
{relevant_chunks}
""".strip()

In [22]:
# 随机抽取10个问题作为测试用例并打印ID
random.seed(42)  # 设置随机种子以确保结果可重复 / Set random seed for reproducibility
test_queries = random.sample(queries, 10)
print("Sample test queries:")
for q in test_queries:
    print(q)

Sample test queries:
賈府為籌建省親別院時，整體殿宇與景觀的規劃是由哪位自號「山子野」的人負責統籌？
在第二十回〈王熙鳳正言彈妒意　林黛玉俏語謔嬌音〉的情節裡，李嬤嬤在屋裡大罵誰「忘了本的小娼婦」，又說她「妝狐媚子哄寶玉」還要「拉出去配一個小子」？
榮國府內眾姐妹居住遊賞的園林名為何？
根據第六十回的敘事，探春如何處置趙姨娘與芳官的衝突？請概述她採取的步驟，並指出這些做法所體現的兩項管理原則。
在王夫人同意她們出家之後，園中戲班的三個女孩分頭投師：其中芳官去了哪座庵、拜哪位師父？另外兩人又分別投向哪座庵、拜哪位師父？
大觀園中因香露與補品失蹤而起紛爭時，誰提出先顧及相關人面子、暫把五兒交由上夜人看守，隔日再向二奶奶稟明處置？
探春與姊妹起詩社，頭一社便不齊全造成混亂，眾人請那位人物出主意處罰主因者？她提出的具體罰則是什麼？
依據第四十四回的敘述：鮑二媳婦吊死一事鬧起來後，賈璉為平息此事最後『許了──發送才罷』。請問他答應發送的銀兩數目是多少？
在王熙鳳病危時，她在三更到四更之間反覆喊著要去哪裡並要做什麼事？
賈府出身、在宮中的那位妃嬪病篤後最終辭世時的年齡是多少？


# Simple Baseline
直接把问题丢给模型，不用知识库

In [23]:
responses = []
# Generate Responses for Each Query 針對每個查詢產生回應
for query in tqdm(test_queries, desc="Generating responses"):
    chats = [
        {'role': "system", 'content': SIMPLE_SYSTEM_PROMPT},
        {'role': "user", 'content': query},
    ]
    response = llm_pipe(chats)[0]['generated_text'][-1]['content'].strip()
    responses.append(response[:512])

# 按顺序打印问答内容
for i, query in enumerate(test_queries):
    print(f"Q: {query}")
    print(f"A: {responses[i]}")
    print("-" * 50)

Generating responses:   0%|          | 0/10 [00:00<?, ?it/s]

Q: 賈府為籌建省親別院時，整體殿宇與景觀的規劃是由哪位自號「山子野」的人負責統籌？
A: 賈府籌建省親別院時，整體殿宇與景觀的規劃由**尤二公**負責統籌。
--------------------------------------------------
Q: 在第二十回〈王熙鳳正言彈妒意　林黛玉俏語謔嬌音〉的情節裡，李嬤嬤在屋裡大罵誰「忘了本的小娼婦」，又說她「妝狐媚子哄寶玉」還要「拉出去配一個小子」？
A: 李嬤嬤所指的「忘了本的小娼婦」指的就是**王熙鳳**。

她之所以罵王熙鳳，是因為王熙鳳在正言行中，冒혔林黛玉的立場，用花言巧語試圖接近寶玉，並暗示黛玉的不足，這被認為是她不顧事實、欺騙世人的表現。

“妝狐媚子哄寶玉”是她藉由妝容和言談，試圖引誘寶玉對黛玉產生偏見，暗示其行為不真誠。

“拉出去配一個小子”則是暗示她希望王熙鳳嫁給一個出身低微的少年，藉此提升自己在賈府的地位和影響力，顯然不滿意王熙鳳的婚姻。
--------------------------------------------------
Q: 榮國府內眾姐妹居住遊賞的園林名為何？
A: 榮國府內眾姐妹居住遊賞的園林主要是**怡紅院、蘅太院**及其他小院。
--------------------------------------------------
Q: 根據第六十回的敘事，探春如何處置趙姨娘與芳官的衝突？請概述她採取的步驟，並指出這些做法所體現的兩項管理原則。
A: 探春處理趙姨娘與芳官衝突的步驟如下：

1.  **親自調解：** 探春先是親自前往芳官居住處，與二人的衝突現場進行調解。
2.  **分別聽取：** 探春先分別聽取了趙姨娘和芳官的陳述，了解事件的起因和各方說法。
3.  **以「產女」為籌碼：** 探春巧妙地利用芳官年紀老、負擔重、且觀察到趙姨娘口氣非但沒有變軟，反而愈來愈尖銳，以「產女」為條件，請求芳官暫時息事。
4.  **告知元春：** 探春將事件告知元春，並建議，如果芳官能產下女兒，趙姨娘便會以「是憂事」為由，先退事。

**這些做法體現的兩項管理原則：**

1.  **以情動人：** 探春巧妙地運用「產女」這種道德和倫理的考量，將問題轉化為一種情感化的期望，以此
-----------------------------------------

# Medium Baseline
只使用 BM25 找前 10 個段落，給模型參考

In [24]:
responses = []
# Generate Responses for Each Query 針對每個查詢產生回應
for query in tqdm(test_queries, desc="Generating responses"):
    relevant_chunks = "\n\n".join(bm25_retrieve(query=query, chunks=chunks, bm25=bm25)[:10])
    chats = [
        {'role': "system", 'content': SYSTEM_PROMPT},
        {'role': "user", 'content': USER_PROMPT.format(query=query, relevant_chunks=relevant_chunks)},
    ]
    response = llm_pipe(chats)[0]['generated_text'][-1]['content'].strip()
    responses.append(response[:512])

# 按顺序打印问答内容
for i, query in enumerate(test_queries):
    print(f"Q: {query}")
    print(f"A: {responses[i]}")
    print("-" * 50)

Generating responses:   0%|          | 0/10 [00:00<?, ?it/s]

Q: 賈府為籌建省親別院時，整體殿宇與景觀的規劃是由哪位自號「山子野」的人負責統籌？
A: 根據您提供的《紅樓夢》片段，與問題“賈府為籌建省親別院時，整體殿宇與景觀的規劃是由哪位自號「山子野」的人負責統籌？”相關的內容是：

“全虧一個老明公號山子野者，一一籌畫起造。”

這句話明確指出“山子野”是負責整體殿宇與景觀規劃的統籌人。
--------------------------------------------------
Q: 在第二十回〈王熙鳳正言彈妒意　林黛玉俏語謔嬌音〉的情節裡，李嬤嬤在屋裡大罵誰「忘了本的小娼婦」，又說她「妝狐媚子哄寶玉」還要「拉出去配一個小子」？
A: 好的，根據您提供的問題和《紅樓夢》片段，我會仔細分析並提供精簡準確的解答。

**問題：** 在第二十回〈王熙鳳正言彈妒意　林黛玉俏語謔嬌音〉的情節裡，李嬤嬤在屋裡大罵誰「忘了本的小娼婦」，又說她「妝狐媚子哄寶玉」還要「拉出去配一個小子」？

**解答：**

根據提供的文章片段，李嬤嬤主要對襲人進行責罵。她以「忘了本的小娼婦」之言，指責襲人專心迎合王熙鳳，以「妝狐媚子哄寶玉」來形容襲人行為，並提出「拉出去配一個小子」的無理要求，表明對襲人行為的不滿和鄙夷。此時的李嬤嬤，確實將襲人視為利用，並對其價值不認。
--------------------------------------------------
Q: 榮國府內眾姐妹居住遊賞的園林名為何？
A: 問題：榮國府內眾姐妹居住遊賞的園林名為何？

答案：

根據提供的《紅樓夢》片段，榮國府內眾姐妹居住遊賞的園林名叫“大觀園”。 

文章中提到：“想畢，遂命太監夏守忠到榮國府來下一道諭，命寶釵等只管在園中居住，不可禁約封錮，命寶玉仍隨進去讀書。” 這其中“園中”一詞，直接點明了大觀園的名稱。
--------------------------------------------------
Q: 根據第六十回的敘事，探春如何處置趙姨娘與芳官的衝突？請概述她採取的步驟，並指出這些做法所體現的兩項管理原則。
A: ## 探春處置趙姨娘與芳官的步驟及體現的兩項管理原則

根據第六十回的敘事，探春處理趙姨娘與芳官衝突的步驟如下：

1.  **最初的ząt反應：** 探春起初還不承認有過錯，以“我本身不願犯法違理”來辯解，表現出猶豫和

# Strong Baseline
結合 BM25 + 向量檢索，使用 RRF 融合排序

In [25]:
# Generate Responses for Each Query 針對每個查詢產生回應
for query in tqdm(test_queries, desc="Generating responses"):
    relevant_chunks_bm25 = bm25_retrieve(query=query, chunks=chunks, bm25=bm25)
    relevant_chunks_embedding = embedding_retrieve(
        query=query,
        chunks=chunks,
        chunks_embeddings=chunks_embeddings,
        embedding_model=embedding_model
    )
    relevant_chunks = "\n\n".join(reciprocal_rank_fusion(relevant_chunks_bm25, relevant_chunks_embedding)[:10])

    chats = [
        {'role': "system", 'content': SYSTEM_PROMPT},
        {'role': "user", 'content': USER_PROMPT.format(query=query, relevant_chunks=relevant_chunks)},
    ]
    response = llm_pipe(chats)[0]['generated_text'][-1]['content'].strip()
    responses.append(response[:512])

# 按顺序打印问答内容
for i, query in enumerate(test_queries):
    print(f"Q: {query}")
    print(f"A: {responses[i]}")
    print("-" * 50)

Generating responses:   0%|          | 0/10 [00:00<?, ?it/s]

Q: 賈府為籌建省親別院時，整體殿宇與景觀的規劃是由哪位自號「山子野」的人負責統籌？
A: 根據您提供的《紅樓夢》片段，與問題“賈府為籌建省親別院時，整體殿宇與景觀的規劃是由哪位自號「山子野」的人負責統籌？”相關的內容是：

“全虧一個老明公號山子野者，一一籌畫起造。”

這句話明確指出“山子野”是負責整體殿宇與景觀規劃的統籌人。
--------------------------------------------------
Q: 在第二十回〈王熙鳳正言彈妒意　林黛玉俏語謔嬌音〉的情節裡，李嬤嬤在屋裡大罵誰「忘了本的小娼婦」，又說她「妝狐媚子哄寶玉」還要「拉出去配一個小子」？
A: 好的，根據您提供的問題和《紅樓夢》片段，我會仔細分析並提供精簡準確的解答。

**問題：** 在第二十回〈王熙鳳正言彈妒意　林黛玉俏語謔嬌音〉的情節裡，李嬤嬤在屋裡大罵誰「忘了本的小娼婦」，又說她「妝狐媚子哄寶玉」還要「拉出去配一個小子」？

**解答：**

根據提供的文章片段，李嬤嬤主要對襲人進行責罵。她以「忘了本的小娼婦」之言，指責襲人專心迎合王熙鳳，以「妝狐媚子哄寶玉」來形容襲人行為，並提出「拉出去配一個小子」的無理要求，表明對襲人行為的不滿和鄙夷。此時的李嬤嬤，確實將襲人視為利用，並對其價值不認。
--------------------------------------------------
Q: 榮國府內眾姐妹居住遊賞的園林名為何？
A: 問題：榮國府內眾姐妹居住遊賞的園林名為何？

答案：

根據提供的《紅樓夢》片段，榮國府內眾姐妹居住遊賞的園林名叫“大觀園”。 

文章中提到：“想畢，遂命太監夏守忠到榮國府來下一道諭，命寶釵等只管在園中居住，不可禁約封錮，命寶玉仍隨進去讀書。” 這其中“園中”一詞，直接點明了大觀園的名稱。
--------------------------------------------------
Q: 根據第六十回的敘事，探春如何處置趙姨娘與芳官的衝突？請概述她採取的步驟，並指出這些做法所體現的兩項管理原則。
A: ## 探春處置趙姨娘與芳官的步驟及體現的兩項管理原則

根據第六十回的敘事，探春處理趙姨娘與芳官衝突的步驟如下：

1.  **最初的ząt反應：** 探春起初還不承認有過錯，以“我本身不願犯法違理”來辯解，表現出猶豫和

# Boss Baseline
使用更多段落

In [26]:
# Generate Responses for Each Query 針對每個查詢產生回應
for query in tqdm(test_queries, desc="Generating responses"):
    relevant_chunks_bm25 = bm25_retrieve(query=query, chunks=chunks, bm25=bm25)
    relevant_chunks_embedding = embedding_retrieve(
        query=query,
        chunks=chunks,
        chunks_embeddings=chunks_embeddings,
        embedding_model=embedding_model
    )
    relevant_chunks = "\n\n".join(reciprocal_rank_fusion(relevant_chunks_bm25, relevant_chunks_embedding)[:20])

    chats = [
        {'role': "system", 'content': SYSTEM_PROMPT},
        {'role': "user", 'content': USER_PROMPT.format(query=query, relevant_chunks=relevant_chunks)},
    ]
    response = llm_pipe(chats)[0]['generated_text'][-1]['content'].strip()
    responses.append(response[:512])

# 按顺序打印问答内容
for i, query in enumerate(test_queries):
    print(f"Q: {query}")
    print(f"A: {responses[i]}")
    print("-" * 50)

Generating responses:   0%|          | 0/10 [00:00<?, ?it/s]

Q: 賈府為籌建省親別院時，整體殿宇與景觀的規劃是由哪位自號「山子野」的人負責統籌？
A: 根據您提供的《紅樓夢》片段，與問題“賈府為籌建省親別院時，整體殿宇與景觀的規劃是由哪位自號「山子野」的人負責統籌？”相關的內容是：

“全虧一個老明公號山子野者，一一籌畫起造。”

這句話明確指出“山子野”是負責整體殿宇與景觀規劃的統籌人。
--------------------------------------------------
Q: 在第二十回〈王熙鳳正言彈妒意　林黛玉俏語謔嬌音〉的情節裡，李嬤嬤在屋裡大罵誰「忘了本的小娼婦」，又說她「妝狐媚子哄寶玉」還要「拉出去配一個小子」？
A: 好的，根據您提供的問題和《紅樓夢》片段，我會仔細分析並提供精簡準確的解答。

**問題：** 在第二十回〈王熙鳳正言彈妒意　林黛玉俏語謔嬌音〉的情節裡，李嬤嬤在屋裡大罵誰「忘了本的小娼婦」，又說她「妝狐媚子哄寶玉」還要「拉出去配一個小子」？

**解答：**

根據提供的文章片段，李嬤嬤主要對襲人進行責罵。她以「忘了本的小娼婦」之言，指責襲人專心迎合王熙鳳，以「妝狐媚子哄寶玉」來形容襲人行為，並提出「拉出去配一個小子」的無理要求，表明對襲人行為的不滿和鄙夷。此時的李嬤嬤，確實將襲人視為利用，並對其價值不認。
--------------------------------------------------
Q: 榮國府內眾姐妹居住遊賞的園林名為何？
A: 問題：榮國府內眾姐妹居住遊賞的園林名為何？

答案：

根據提供的《紅樓夢》片段，榮國府內眾姐妹居住遊賞的園林名叫“大觀園”。 

文章中提到：“想畢，遂命太監夏守忠到榮國府來下一道諭，命寶釵等只管在園中居住，不可禁約封錮，命寶玉仍隨進去讀書。” 這其中“園中”一詞，直接點明了大觀園的名稱。
--------------------------------------------------
Q: 根據第六十回的敘事，探春如何處置趙姨娘與芳官的衝突？請概述她採取的步驟，並指出這些做法所體現的兩項管理原則。
A: ## 探春處置趙姨娘與芳官的步驟及體現的兩項管理原則

根據第六十回的敘事，探春處理趙姨娘與芳官衝突的步驟如下：

1.  **最初的ząt反應：** 探春起初還不承認有過錯，以“我本身不願犯法違理”來辯解，表現出猶豫和

# Create Prediction File

In [27]:
# 輸出回應結果為 JSON 檔案
output_path = os.path.join(OUTPUT_DIR, "hw2.json")
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(responses, f, ensure_ascii=False, indent=2)